# --- Day 8: Seven Segment Search --- 

https://adventofcode.com/2021/day/8

## Get Input Data

In [1]:
def parse_data(filename):
    with open(f'../inputs/{filename}') as file:
        data = []
        for line in file:
            data.append([x.split(' ') for x in line.strip().split(' | ')])
    return data

In [2]:
test_data = parse_data('test_sources_outputs.txt')
test_data[:2]

[[['be',
   'cfbegad',
   'cbdgef',
   'fgaecd',
   'cgeb',
   'fdcge',
   'agebfd',
   'fecdb',
   'fabcd',
   'edb'],
  ['fdgacbe', 'cefdb', 'cefbgd', 'gcbe']],
 [['edbfga',
   'begcd',
   'cbg',
   'gc',
   'gcadebf',
   'fbgde',
   'acbgfd',
   'abcde',
   'gfcbed',
   'gfec'],
  ['fcgedb', 'cgb', 'dgebacf', 'gc']]]

In [3]:
data = parse_data('sources_outputs.txt')
data[:2]

[[['ecdbfag',
   'deacfb',
   'acdgb',
   'cdg',
   'acdbf',
   'gdfb',
   'efacdg',
   'gd',
   'cagdbf',
   'beacg'],
  ['cdg', 'dcebgaf', 'gbdf', 'bdacg']],
 [['fadecg',
   'gdbecaf',
   'agbfd',
   'fgdcb',
   'gab',
   'ebagdf',
   'feabcg',
   'deab',
   'gdefa',
   'ab'],
  ['adfbg', 'ab', 'fcgdbae', 'bfgecda']]]

## Part 1
---

In [4]:
def count_easy_digits(data):
    """Count the easy digits (known solely based on their character length) in the output data."""
    
    num_easy_digits = 0
    
    for row in data:
        outputs = data[1]
        
        for o in outputs:
            if len(o) in [2, 3, 4, 7]:
                num_easy_digits += 1
    
    return num_easy_digits

### Run on Test Data

In [5]:
count_easy_digits(test_data)  # Should return 26

10

### Run on Input Data

In [6]:
count_easy_digits(data)

200

## Part 2
---

In [7]:
test_signals = ['acedgfb', 'cdfbe', 'gcdfa', 'fbcad', 'dab', 'cefabd', 'cdfgeb', 'eafb', 'cagedb', 'ab']

In [8]:
def make_map(signals):
    """Make a map of all the numbers represented by the 10 seven-segment character codes."""    
    
    # Make a copy of signals (just being lazy and not importing copy)
    signals = [x for x in signals]

    letter_map = {}
    letter_map[[x for x in signals if len(x) == 2][0]] = '1'
    letter_map[[x for x in signals if len(x) == 3][0]] = '7'
    letter_map[[x for x in signals if len(x) == 4][0]] = '4'
    letter_map[[x for x in signals if len(x) == 7][0]] = '8'
    
    number_map = {}
    for k, v in letter_map.items():
        number_map[int(v)] = k
        signals.remove(k)
        
    # Get #9 (#9... #9... #9...)
    top = list(set([char for char in number_map[7]]) - set([char for char in number_map[1]]))[0]
    temp = number_map[4] + top
    # ^ means 'symmetric difference' -> in s or temp, but not both
    bottom = [list(x)[0] for x in [set(s) ^ set(temp) for s in signals] if len(x) == 1][0]
    nine = [x for x in signals if set(x) == set(temp + bottom)][0]
    
    letter_map[nine] = '9'
    number_map[9] = nine
    signals.remove(nine)
    
    # Get #3
    temp = number_map[7] + bottom
    middle = [list(x)[0] for x in [set(s) ^ set(temp) for s in signals] if len(x) == 1][0]
    three = [x for x in signals if set(x) == set(temp + middle)][0]
    
    letter_map[three] = '3'
    number_map[3] = three
    signals.remove(three)
    
    # Get #0
    top_left = list(set(number_map[9]) ^ set(number_map[3]))[0]
    bottom_left = list(set(number_map[8]) ^ set(number_map[9]))[0]
    temp = number_map[7] + top_left + bottom_left + bottom
    zero = [x for x in signals if set(x) == set(temp)][0]
    
    letter_map[zero] = '0'
    number_map[0] = zero
    signals.remove(zero)
    
    # Get #6
    six = [x for x in signals if len(x) == 6][0]
    
    letter_map[six] = '6'
    number_map[6] = six
    signals.remove(six)
    
    # Get #5
    five = [x for x in signals if len(set(x) ^ set(number_map[6])) == 1][0]
    
    letter_map[five] = '5'
    number_map[5] = five
    signals.remove(five)
    
    # Get #2
    two = signals[0]
    
    letter_map[two] = '2'
    number_map[2] = two
    signals.remove(two)
    
    return letter_map

In [9]:
make_map(test_signals)

{'ab': '1',
 'dab': '7',
 'eafb': '4',
 'acedgfb': '8',
 'cefabd': '9',
 'fbcad': '3',
 'cagedb': '0',
 'cdfgeb': '6',
 'cdfbe': '5',
 'gcdfa': '2'}

In [10]:
def decode_number(outputs, map):
    """Decode number based on map."""
    
    number = ''
    
    for code in outputs:
        # Ugh. codes are jumbled too...
        for key in map.keys():
            if set(code) == set(key):
                number += map[key]
        
    return int(number)

In [11]:
def sum_numbers(data):
    """Return tue sum of all the encoded numbers in the outputs."""
    total = 0
    
    for row in data:
        signals = row[0]
        outputs = row[1]
        
        letter_map = make_map(signals)
        number = decode_number(outputs, letter_map)
        
        total += number
    
    return total

### Run on Test Data

In [12]:
sum_numbers(test_data)  # Should return 61229

61229

### Run on Input Data

In [13]:
sum_numbers(data)

1083859